In [1]:
import pandas as pd, numpy as np, json
from glob import glob

In [2]:
glob('*')

['daily.csv', 'hourly.csv', 'read_data.ipynb', 'stations.csv', 'weekdays.csv']

In [3]:
stations = pd.read_csv('stations.csv')

stations.columns = ['id', 'kpas', 'lon', 'lat', 'name']
stations.kpas = stations.kpas.round(0).astype(int)

In [4]:
# stations.to_json('stations.json', orient='records', force_ascii=False)

In [5]:
hourly = pd.read_csv('hourly.csv')
weekdays = pd.read_csv('weekdays.csv')
daily = pd.read_csv('daily.csv')

In [6]:
daily.kpas = daily.kpas.round().astype(int)
daily = pd.pivot_table(daily, index=['station', 'date', 'name'], columns='type', values='kpas').reset_index()
daily.date = pd.to_datetime(daily.date)

In [7]:
year = pd.DataFrame(pd.DatetimeIndex(freq=pd.DateOffset(days=1), start='2014-01-01', end='2014-12-31'), columns=['date'])

In [18]:
def get_st(st, name, lon, lat, kpas):
    st_daily = daily.loc[daily.station == st, ['date','arrival','departure']
                   ].merge(year, how='right'
                   ).fillna(0
                   ).sort_values('date')
    st_daily.date = st_daily.date.astype(str)
    st_daily = st_daily.to_dict(orient='records')
    
    st_weekdays = weekdays.loc[weekdays.station == st, ['weekday', 'arrival', 'departure']
                         ].round(2
                         ).fillna(0
                         ).to_dict(orient='records')
    st_hourly = hourly.loc[hourly.station == st, ['hour', 'arrival', 'departure']
                     ].round(2
                     ).fillna(0
                     ).to_dict(orient='records')
    d = {
        'name': name,
        'id': st,
        'lon': lon,
        'lat': lat,
        'kpas': kpas,
        'daily': st_daily,
        'weekdays': st_weekdays,
        'hourly': st_hourly,
    }
    return st, d

all_st = dict(get_st(*t) for t in stations[['id', 'name', 'lon', 'lat', 'kpas']].values)

In [19]:
with open('../src/assets/data/all_stations.json', 'w', encoding='utf-8') as f:
    json.dump(all_st, f, ensure_ascii=False)